Importar librerias libreriales

In [2]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

print('Keras version: ',keras.__version__)

Using TensorFlow backend.
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

Keras version:  2.3.0


Importar dataset de Bruno

In [168]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [169]:
# lectura del dataset
df = pd.read_csv("3x3-a-train.csv")
y_train = df['init_move']
X_train = df.drop(labels=['init_move', 'final_move'], axis='columns')
df = pd.read_csv("3x3-a-test.csv")
y_test = df['init_move']
X_test = df.drop(labels=['init_move', 'final_move'], axis='columns')
X_train = X_train.to_numpy()
X_train2 = np.delete(X_train,[0,10,11,12,13,14],1)
y_train = y_train.to_numpy()
X_train_reshaped=np.expand_dims(X_train2, axis=2)
print(X_train.shape,X_train_reshaped.shape, y_train.shape)
#print(X_train.head(10))
#print(y_train.head(10))
# Reshape data 
#input_train = X_train.reshape( X_train.shape[0], X_train.shape[1])
#input_test  = input_test.reshape(len(input_test), input_shape[0], input_shape[1], input_shape[2])

(10933, 15) (10933, 9, 1) (10933,)


In [170]:
X_train_reshaped.shape = (10933,3,3)
X_train_reshaped[0]

array([[0.33333333, 0.11111111, 0.44444444],
       [0.55555556, 0.        , 0.        ],
       [0.33333333, 0.22222222, 0.        ]])

In [190]:
#Aquí se obtinenen los atributos "no vectoriales"
XX_train =X_train[0:,10:15]
XX_train.shape

(10933, 5)

In [191]:
y_train[:10]

array([0, 1, 2, 2, 1, 1, 2, 2, 0, 0], dtype=int64)

In [192]:
np.unique(y_train)

array([0, 1, 2], dtype=int64)

In [193]:
from sklearn.preprocessing import OneHotEncoder

In [194]:
encoder = OneHotEncoder()
y_train_coded = encoder.fit_transform(y_train.reshape(-1,1)).toarray() 

In [195]:
y_train_coded[:10]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

Modelado de la red

In [3]:
convmodel = Sequential()
convmodel.add(Conv1D(filters=30, kernel_size=2, activation='relu', input_shape=(5,1)))
convmodel.add(Conv1D(filters=30, kernel_size=1, activation='relu'))
convmodel.add(Dropout(0.1))
convmodel.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(3, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
convmodel.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 4, 30)             90        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 30)             930       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 30)             0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2, 30)             0         
Total params: 1,020
Trainable params: 1,020
Non-trainable params: 0
_________________________________________________________________


In [208]:
#define two different inputs
input1 = Input((3, 1))
input2 = Input((3, 1))
input3 = Input((3, 1))
input4 = Input((5, ))
   

#use the model to get three different outputs (the same model)
out1 = convmodel(input1)
out2 = convmodel(input2)
out3 = convmodel(input3)

#concatenate the outputs and add the final part of your model: 
out = Concatenate()([out1,out2,out3])
out = Flatten()(out)
out = Concatenate()([out,input4])
out = Dense(20, activation='relu')(out)
out = Dense(3, activation='softmax')(out)   

#create the model taking 2 inputs with one output
model = Model([input1,input2,input3,input4],out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.save_weights('model.h5')
model.summary()


Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_139 (InputLayer)          (None, 3, 1)         0                                            
__________________________________________________________________________________________________
input_140 (InputLayer)          (None, 3, 1)         0                                            
__________________________________________________________________________________________________
input_141 (InputLayer)          (None, 3, 1)         0                                            
__________________________________________________________________________________________________
sequential_21 (Sequential)      (None, 1, 30)        1020        input_139[0][0]                  
                                                                 input_140[0][0]           

In [211]:
X_train1 = X_train_reshaped[0:,0]
X_train2 = X_train_reshaped[0:,1]
X_train3 = X_train_reshaped[0:,2]
X_train1.shape=(10933,3,1)
X_train2.shape=(10933,3,1)
X_train3.shape=(10933,3,1)

model.load_weights('model.h5')
model.fit([X_train1,X_train2,X_train3,XX_train], y_train_coded, epochs=100, batch_size=64, verbose=True)

Epoch 1/100
10933/10933 [==============================] - 1s 57us/step - loss: 0.8437 - accuracy: 0.6126
Epoch 2/100
10933/10933 [==============================] - 1s 58us/step - loss: 0.6033 - accuracy: 0.7579
Epoch 3/100
10933/10933 [==============================] - 1s 50us/step - loss: 0.5293 - accuracy: 0.7878
Epoch 4/100
10933/10933 [==============================] - 1s 53us/step - loss: 0.4911 - accuracy: 0.8036
Epoch 5/100
10933/10933 [==============================] - 1s 51us/step - loss: 0.4724 - accuracy: 0.8086
Epoch 6/100
10933/10933 [==============================] - 1s 53us/step - loss: 0.4608 - accuracy: 0.8143
Epoch 7/100
10933/10933 [==============================] - 1s 54us/step - loss: 0.4469 - accuracy: 0.8178
Epoch 8/100
10933/10933 [==============================] - 1s 56us/step - loss: 0.4342 - accuracy: 0.8259
Epoch 9/100
10933/10933 [==============================] - 1s 60us/step - loss: 0.4207 - accuracy: 0.8350
Epoch 10/100
10933/10933 [====================